# ESMDA for parameter estimation in microscale pollutant dispersion 

This repository provides an example of Python implementation of the Ensemble Smoother with Multiple Data Assimilation (ESMDA) [1] algorithm. In this example, the algorithm is used to estimate meteorological forcing parameters (wind direction and friction velocity) using pollutant concentration measurements. The ESMDA thereby improve the accuracy and robustness of a POD--GPR surrogate model of pollutant dispersion in a simplified urban environment. 

#### Imports

In [3]:
import time
import numpy as np

from models import MustSurrogate
from auxiliaries import log_transform_cut, halton_multivariate_truncated_normal
from metrics import nmse, fac2, vg

#### Main inputs

In [ ]:
# ESMDA parameters
Na = 4                                              # Number of assimilations
Ne = 500                                            # Number of members
c_anamorphosis = 0.04                               # Threshold used in the concentration anamorphosis

# Model input parameters
alpha_true = -40.95                                 # Reference inlet wind direction (°) (truth)
ustar_true = 0.730                                  # Reference friction velocity (m/s) (truth)
alpha_b = -25                                       # Background inlet wind direction (°) (first guess)
ustar_b = 0.57                                      # Background friction velocity (m/s) (first guess)
sigma_b_alpha = 25.                                 # Background inlet wind direction error std (°) (uncertainty)
sigma_b_ustar = 0.09                                # Background friction velocity error std (m/s) (uncertainty)

# Model loading
model = MustSurrogate(alpha_b, ustar_b)

# Observation network selection
obs_indexes = [33, 34, 35, 37, 38, 39, 43, 44, 45]  # UVIC sensors at z=1,2,3m at towers B, C, D

# Random generator initialization
seed_osse = 41                                      # Seed used to generate the idealized observations
seed_obs = 31415                                    # Seed used to perturbate the observations in the ESMDA
seed_rom = 66261                                    # Seed used to get the random realization of the POD-GPR model
rng_osse = np.random.default_rng(seed_osse)         # Initialize random number generator for reproducibility
rng_esmda = np.random.default_rng(seed_obs)

#### Idealized observations generation
In the following, idealized obserations are generated from a reference prediction of the forward model (the 'truth'). This is done by adding a random noise, representative of the real-world observation error, to the predicted tracer concentration at observed location. This provides a simplified framework, called twin experiment or observing system simulation experiment [2], to verify that the ESMDA algorithm works properly. 

In [ ]:
# Reference time-averaged concentration field prediction (truth)
model.set_parameters(alpha_true, ustar_true)
model.forecast()
x_true = model.get_avg_state()

# Mesh reading
mesh_nodes = model.get_mesh_nodes()
mesh_conns = model.get_mesh_conns()

# Observation operator definition
probes_network = np.loadtxt('data/probes_network.dat', skiprows=1, usecols=(2,3,4))
Np = len(obs_indexes)
obs_nodes = np.zeros(Np, dtype='int')
for i, k in enumerate(obs_indexes):  # Search the nodes the closest to the assimilated probes
    distances_to_probe = np.sqrt((mesh_nodes[:,0] - probes_network[k,0])**2 \
                                  + (mesh_nodes[:,1] - probes_network[k,1])**2 \
                                  + (mesh_nodes[:,2] - probes_network[k,2])**2)
    obs_nodes[i] = np.argmin(distances_to_probe)

def H(x, obs_nodes=obs_nodes):  
    """
    Observation operator
    """
    return x[obs_nodes]

#TODO: remove
probes_names = np.loadtxt('data/probes_network.dat', skiprows=1, usecols=(1), dtype='str')
print(probes_names[obs_indexes])

# Load the orecomputed observation error covariance matrix 
R = np.load('../../obs_data/assim_obs/obs_log_error_covariance_matrix.npy')
R = R[np.ix_(obs_indexes, obs_indexes)]
#   This matrix accounts for the instrument measurement error and the uncertainty 
#   related to the internal variability of the atmospheric boundary layer.

# Synthetic observations generation
obs_true = H(x_true)
obs_log = log_transform_cut(obs_true, c_anamorphosis)  # Concentration anamoprhosis ln(max(y,0) + c_anamorphosis)
y = rng_osse.multivariate_normal(obs_log, R)            # Perturbations to not exactly match the true concentration field

#### ESMDA Initialization
Sample the background ensemble, which represents the probabilistic prior estimate of the meteorological forcing parameters.

In [ ]:
# Dimension and ensemble initialization
Np = 2                                       # Parameter space dimension 
Ns = np.shape(mesh_nodes)[0]                 # State space dimension
No = np.shape(y)[0]                          # Observation space dimension
Eb = np.zeros((Ne, Np))                      # Background ensemble
Ea = np.zeros((Ne, Np))                      # Analysis ensemble vector
E_xb = np.zeros((Ne, Ns))                    # Background state ensemble
H_E_xb = np.zeros((Ne, No))                  # Background ensemble vector in the observation space

# Coefficient in the ESMDA
alpha_ESMDA = Na*np.ones(Na)

# Background error covariance matrix
B = np.eye(Np)                                      
B[0,0] = sigma_b_alpha**2
B[1,1] = np.log(1 + sigma_b_ustar**2/ustar_b**2)  # Considering friction velocity anamorphosis

# Compute the unbiased background parameter vector considering the friction velocity anamorphosis
theta_b = np.array([alpha_b, np.log(ustar_b) - 0.5*B[1,1]])

# Background ensemble sampling
# A truncated normal distribution is used to sample each parameter  to ensure that the model gets well defined parameters
# The lower bounds used to truncate the normal distribution before sampling to 
# ensure compliance with the model input parameter limits
lower_bounds = np.array([model._alpha_min, np.log(model._ustar_min)])  
upper_bounds = np.array([model._alpha_max, np.inf])    
Eb0 = halton_multivariate_truncated_normal(theta_b, B, lower_bounds, upper_bounds, Ne)

#### ESMDA parameter estimation algorithm

In [ ]:
time_start = time.time()  # To monitor the ESMDA execution time
print('\n____________________ Data Assimilation Begins ____________________')

for j in range(Na):
    ### A) Prediction
    # Produce an ensemble of perturbed observations
    Y = rng_esmda.multivariate_normal(y, alpha_ESMDA[j]*R, Ne)

    # Compute the mean of the current ensemble
    if j==0:  
        Eb = Eb0.copy()

    else:  
        # We use the analysis of the previous assimilation as background
        if np.any((Ea - lower_bounds)<0) or np.any((Ea - upper_bounds)>0):
            # If one anlysis member is out of the model input parameter variation ranges, we warn the user:    
            print(f"Warning: {np.sum((Ea - lower_bounds)<0) + np.sum((Ea - upper_bounds)>0)} analysis sample(s) is(are) outisde of the model input parameter variation ranges. Truncated re-sampling will be used.")
            
            # We then re-sample an ensemble using a truncated normal distribution based on the analysis ensemble
            Eb = halton_multivariate_truncated_normal(np.mean(Ea, axis=0), 
                                                      np.diag([np.var(Ea[:,0], ddof=1), np.var(Ea[:,1], ddof=1)]),
                                                      lower_bounds, upper_bounds, Ne)
            
        else:
            Eb = Ea.copy()

    # Forecast to obtain the background state ensemble from the background parameters ensemble
    for i in range(Ne):
        model.set_parameters(Eb[i,0], np.exp(Eb[i,1]))
        E_xb[i] = model.sample_y(n_samples=1, random_state=seed_rom+10*(Ne*j+i))[0]  # Change the seed deterministically for each model evaluation

        # Map the ensemble members to the observation space using the observation operator
        H_E_xb[i] = H(E_xb[i]) 

    H_E_xb = log_transform_cut(H_E_xb, c_anamorphosis)  # Use log(max(H(x_b),0) + c_anamorphosis) to compare with obs of log-concentration
    H_E_xb_mean = np.mean(H_E_xb, axis=0)

    ### B) Analysis
    # Compute the Kalman Gain using covariance estimation
    theta_b_mean = np.mean(Eb, axis=0)
    BGt = np.dot((Eb - theta_b_mean).T, H_E_xb - H_E_xb_mean) / (Ne-1)  
    GBGt = np.dot((H_E_xb - H_E_xb_mean).T, H_E_xb - H_E_xb_mean) / (Ne-1)

    K = np.matmul(BGt, np.linalg.inv(GBGt + alpha_ESMDA[j]*R))

    # Compute the analysis of each ensemble member
    for i in range(Ne):
        Ea[i] = Eb[i] + np.matmul(K, Y[i] - H_E_xb[i])  # Analysis update

    print(f'Assimilation #{j+1}/{Na}')

print('\n\n_____________________ Data Assimilation Over _____________________\n')
print(f"Execution time = {time.time() - time_start:.2f}s")

#### Post-processing and evaluation
The accuracy of the ESMDA parameter estimation is evaluated using the standard air quality metrics defined by [3].

In [ ]:
### Inverse anamorphosis and statistics computation 
alpha_a_mean = np.mean(Ea, axis=0)[0]
alpha_a_std = np.std(Ea, axis=0, ddof=1)[0]
ustar_a_mean = np.mean(Ea, axis=0)[1]
ustar_a_std = np.std(Ea, axis=0, ddof=1)[1]

### Print analysis parameters prediction
print(f"Analysis alpha_inlet mean estimation: {alpha_a_mean:.2f}°")
print(f"Analysis ustar mean estimation: {ustar_a_mean:.3f} m/s")
print(f"Analysis alpha_inlet mean estimation error: {alpha_a_mean - alpha_true:.2f}°")
print(f"Analysis ustar mean estimation error: {ustar_a_mean - ustar_true:.3f} m/s")
print(f"Analysis alpha_inlet std deviation estimation: {alpha_a_std:.2f}°")
print(f"Analysis ustar std deviation estimation: {ustar_a_std:.3f} m/s")

# Compute the mean background and analysis state estimations
model.set_parameters(alpha_b, ustar_b)
model.forecast()
xb = model.get_avg_state()
model.set_parameters(alpha_a_mean, ustar_a_mean)
model.forecast()
xa = model.get_avg_state()

# Air quality metrics
nmse_a = nmse(xa, x_true)
nmse_b = nmse(xb, x_true)
fac2_a = fac2(xa, x_true, 1e-4)
fac2_b = fac2(xb, x_true, 1e-4)
vg_a = vg(xa, x_true, 1e-4)
vg_b = vg(xb, x_true, 1e-4)

print(f"\nAir quality metrics scores (compared to the complete truth field):")
print(f"\t *) Background: NMSE = {nmse_b:.2f} | FAC2 = {100*fac2_b:.2f}% | VG = {vg_b:.2f}")
print(f"\t *) Analysis:   NMSE = {nmse_a:.2f} | FAC2 = {100*fac2_a:.2f}% | VG = {vg_a:.2f}")

#### References

[1] Emerick, A. A. and Reynolds, A. C. (2013). Ensemble smoother with multiple data assimilation. Computers & Geosciences, 55:3–15. DOI: [10.1016/j.cageo.2012.03.011](https://doi.org/10.1016/j.cageo.2012.03.011).

[2] Arnold, C. P. and Dey, C. H. (1986). Observing-Systems Simulation Experiments: Past, Present, and Future. Bulletin of the American Meteorological Society, 67(6):687 – 695. DOI: [10.1175/1520-0477(1986)067<0687:OSSEPP>2.0.CO;2](https://doi.org/10.1175/1520-0477(1986)067<0687:OSSEPP>2.0.CO;2).